# Подготовительный этап

## Получение чата

### Подготовительный этап

#### Цели:
1. Получить ***vk token***
2. Получить ***id чата***, состоящий из двух цифр
3. Определиться, должна ли будет проводиться загрузка фото и стикеров

#### Решение:
1. Гугл в помощь + ***.env*** для хранения
2. Адресная строка в помощь
3. Введена дополнительная переменная для определения того, следует ли пытаться загрузить изображения и стикеры, или нет

In [ ]:
import vk_api
import requests as rq
import pandas as pd
from math import ceil
from datetime import datetime

from dotenv import dotenv_values

secrets = dotenv_values(".env")
"""Секреты"""

vk = vk_api.VkApi(
    token=secrets['VK_TOKEN'])
"""Модуль ВК"""

id_chat = int(secrets['CHAT_ID'])
"""id чата"""

SHOULD_DOWNLOAD_PHOTO = False
"""Должна ли проводиться загрузка фото"""
SHOULD_DOWNLOAD_STICKER = False
"""Должна ли проводиться загрузка стикеров"""


### Определение функций

Необходимо определиться с тем, как будет получаться необходимая информация с vk.
Поскольку используется ***vk_api***, то можно использовать официальную документацию по работе с api.

**get_chat** — работает с ***vk.method***, поэтому все параметры приходится прописывать в роли dictionary.
**download_&** — работает с помощью ***urllib3***. Получается изображение с сайта, затем оно сохраняется и готово.

Дополнительную информацию можно прочесть в документации в коде.

In [ ]:
def download_image(url: str) -> None:
    """

    Функция загрузки изображения

    :param url: URL адрес картинки
    :type url: str

    :return: Сохраняет картинку в папку
        Каждая картинка появляется лишь 1 раз
    :rtype: None

    """
    file_name = (url.split("/")[-1]).split("?")[0]
    """Имя картинки"""

    result_image = rq.get(url)
    """Полученное изображение"""
    with open(f'content/visual/images/{file_name}', 'wb') as image:
        image.write(result_image.content)


def download_sticker(id_sticker: int) -> None:
    """

    Функция загрузки стикера

    :param id_sticker: Номер стикера
    :type id_sticker: int

    :return: Сохраняет стикер в папку
        Каждый стикер появляется лишь 1 раз
        Стикеры сохраняются в формате .png в разрешении 512x512
    :rtype: None

    """
    result_sticker = rq.get(f"https://vk.com/sticker/1-{id_sticker}-512b")
    """Полученный стикер"""
    with open(f'content/visual/stickers/{id_sticker}.png', 'wb') as f:
        f.write(result_sticker.content)


def get_chat(peer_id: int = id_chat, count: int = 200, offset: int = 0) -> dict:
    """

    Позволяет получить сообщения из чата.

    :param peer_id: id чата.
        Работает через peer
    :type peer_id: int
    :param count: Количество получаемых сообщений <= 200
    :type count: int
    :param offset: Сдвиг от начального сообщения
    :type offset: int

    :return: Словарь с сообщениями и их параметрами
    :rtype: dict

    """
    # Требуется добавлять 2e9 по документации vk api
    peer_id += 2e9

    return vk.method('messages.getHistory',
                     {'peer_id': peer_id,
                      'count': count,
                      'offset': offset,
                      'rev': 1,
                      'extended': True}
                     )


### Получение чата

Теперь необходимо получить сам чат.

Делается это в два этапа:

#### Этап 1:
##### Получение длины чата.

Берётся одно сообщение из чата.
В каждый ***response*** закладывается количество сообщений в чате.
С его помощью можно завести цикл по количеству сообщений.

#### Этап 2:
##### Сам цикл.

За раз я могу получить лишь 200 сообщений из чата.
***delta*** делает отступ от первого сообщения в чате, а также является счётчиком того, сколько сообщений было прочитано.

#### Этап 3:
##### Получение пользователей.

Каждый раз при запуске нового большого цикла производится добавление пользователей, что так или иначе были задействованы в этих сообщениях (пересланные сообщения и реакции тоже считаются), добавляются в **users_mass**, содержащий **id** пользователей и их **Имя** и **Фамилию**.

Пользователи с удалённым профилем тоже обрабатываются. В качестве **Имени** ***vk*** даёт им *DELETED*, а в качестве **Фамилии** берётся их **id**.

#### Этап 4:
##### Разбор сообщений.

Сообщения имеют следующие характеристики:
1. **id самого сообщения**. id считается относительно всех сообщений в чате
2. **id пользователя**
3. **Является ли действием** *(добавление/исключение пользователей)*
4. **Пересылается ли сообщение в сообщении**
5. **Текст сообщения**
6. **Приложенный файл**
7. **Реакции на сообщение**
8. **Информация о сообщении, на которое даётся ответ**. Необходимо брать эту информацию, поскольку может быть ситуация, когда сообщение, на которое был дан ответ, было позже удалено.

Более подробно о том, как разбираются сообщения будет лучше посмотреть в коде, но если вкратце — берётся разнообразная информация из оригинального сообщения (**message_data**) и добавляется в финальное сообщение (**message**), которое затем добавляется в **msg_mass**.

Удалённые сообщения. Но вместо их **id** берётся константа *404404* и приписывается текущее количество удалённых сообщений.


In [ ]:
length_chat = get_chat(count=1)['count']
"""Количество сообщений в чате"""

msg_mass = []
"""Массив сообщений"""

users_mass = {}
"""Список пользователей"""

count_dead_msg = 0
"""Количество удалённых сообщений, на которые был дан ответ"""

start_time = datetime.now()
"""Время начала получения статистики"""
for times_add in range(int(ceil(length_chat / 200))):

    delta = 200 * times_add
    """Отступ от первого сообщения"""
    messages = get_chat(count=min(200, length_chat - delta), offset=delta)
    """count сообщений после delta"""

    for profile in messages['profiles']:
        if users_mass.get(profile['id']):
            continue
        users_mass[profile['id']] = profile['first_name'] + " "
        if profile['last_name'] == '':
            users_mass[profile['id']] += str(profile['id'])
        else:
            users_mass[profile['id']] += profile['last_name']

    for message_data in messages['items']:
        isForwarding = True if message_data.get("fwd_messages") else False
        """Пересылается ли сообщение"""

        isAction = True if message_data.get('action') else False
        """Является ли сообщение действием"""

        attachments_type = "None"
        """Тип прикреплённого сообщения"""
        attachments = []
        """Прикреплённые доп. материалы"""
        if message_data.get('attachments'):

            for attachment in message_data['attachments']:
                if attachment['type'] == 'photo':
                    attachments_type = 'photo'
                    attachments.append(
                        (((attachment['photo']['sizes'][-1]['url']).split("/")[-1]).split("?"))[0]
                    )
                elif attachment['type'] == 'sticker':
                    attachments_type = 'sticker'
                    attachments.append(str(attachment['sticker']['sticker_id']) + ".png")

        reactions = [0] * (16 + 1)
        """Реакции"""
        if message_data.get('reactions'):
            # Почему-то не всегда показывает тех, кто ставил реакции
            # #1
            for reaction in message_data['reactions']:
                reactions[0] += reaction['count']
                user_list = [reaction['count']]
                for user in reaction['user_ids']:
                    user_list.append(user)
                reactions[reaction['reaction_id']] = user_list

        response = {'id': -1,
                    'date': -1,
                    'user_id': -1,
                    'text': "None",
                    'attachments': {'type': "None",
                                    'value': []
                                    # Содержит в себе название файла
                                    }
                    }
        """Ответ на сообщение"""
        if message_data.get('reply_message'):
            reply = message_data['reply_message']
            if reply.get('conversation_message_id'):
                response['id'] = reply['conversation_message_id']
            if not reply.get('conversation_message_id') or response['id'] is None:
                response['id'] = int(f'404404{count_dead_msg}')
                count_dead_msg += 1
            response['date'] = reply['date']
            response['user_id'] = reply['from_id']
            response['text'] = reply['text']

            for attachment in reply['attachments']:
                if attachment['type'] == 'photo':
                    response['attachments']['type'] = 'photo'
                    response['attachments']['value'].append(
                        (((attachment['photo']['sizes'][-1]['url']).split("/")[-1]).split("?"))[0]
                    )
                elif attachment['type'] == 'sticker':
                    response['attachments']['type'] = 'sticker'
                    response['attachments']['value'].append(str(attachment['sticker']['sticker_id']) + ".png")

        message = {'id': message_data['conversation_message_id'],
                   'date': message_data['date'],
                   'isAction': isAction,
                   'isForwarding': isForwarding,
                   'id_user': message_data['from_id'],
                   'text': message_data['text'],
                   'attachments_type': attachments_type,
                   'attachments': attachments,
                   'reactions': reactions,
                   'response_id': response['id'],
                   'response_date': response['date'],
                   'response_id_user': response['user_id'],
                   'response_text': response['text'],
                   'response_attachments_type': response['attachments']['type'],
                   'response_attachments': response['attachments']['value']
                   }
        """Сообщение"""

        msg_mass.append(message)

        # Загрузка доп данных
        if any((SHOULD_DOWNLOAD_PHOTO, SHOULD_DOWNLOAD_STICKER)) and message_data.get('attachments'):
            for attachment in message_data['attachments']:
                if attachment['type'] == 'photo' and SHOULD_DOWNLOAD_PHOTO:
                    download_image(attachment['photo']['sizes'][-1]['url'])
                elif attachment['type'] == 'sticker' and SHOULD_DOWNLOAD_STICKER:
                    download_sticker(attachment['sticker']['sticker_id'])

end_time = datetime.now()
"""Время завершения программы получения статистики"""
print(end_time - start_time)

users_df = pd.DataFrame(users_mass.items(), columns=['id', 'username'])
chat_df = pd.DataFrame(msg_mass)

users_df

In [ ]:
chat_df

Проверка на правильность вводимых типов данных.

Некоторые из них помечены как *object*. Это *str*, *dict*, *list* и им подобные структуры данных.

In [ ]:
chat_df.dtypes.sort_values()

## Сохранение
Сохраним данные, чтобы, при получении статистики в несколько этапов, не приходилось каждый раз её генерировать.

In [ ]:
chat_df.to_csv("content/History.csv")
users_df.to_csv("content/Users.csv")

# Статистика

## Загрузка
Процесс, обратный сохранению.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
chat_df = pd.read_csv("content/History.csv", index_col="Unnamed: 0")
users_df = pd.read_csv("content/Users.csv", index_col="Unnamed: 0")

In [ ]:
to_transform_columns = ['attachments', 'reactions', 'response_attachments']
"""Колонки, которые необходимо привести к нормальному виду"""
for column in to_transform_columns:
    chat_df[column] = chat_df[column].apply(lambda x: eval(x))

In [ ]:
chat_df

In [ ]:
users_df

## Статистика

In [ ]:
import matplotlib.pyplot as plt
import pytz
from datetime import datetime
import matplotlib.image as mpimg

Введём дополнительную функцию, позволяющую перевести время из *UTC* в *YYYY-MM-DD HH:MM:SS*

In [ ]:
def get_date(utc_date: int, ymd: bool = True, h: bool = True) -> str:
    """

    Перевод даты из UTC формата в нормальный формат

    :param h: Должны ли быть выведены Часы?
    :type h: bool
    :param ymd: Должны ли быть выведены Год Месяц День?
    :type ymd: bool
    :param utc_date: Дата в формате utc
    :type utc_date: int
    
    :exception: Можно или запросить всё целиком, или только год-месяц-день, или только часы.
        Нельзя запросить ничего!

    :return: Дата в формате YYYY-MM-DD HH:MM:SS
    :rtype: str

    """
    timezone = pytz.timezone("Asia/Vladivostok")
    """Ваша временная зона"""
    date = datetime.fromtimestamp(utc_date, tz=timezone)
    """Дата без форматирования"""
    if ymd and h:
        return date.strftime('%Y-%m-%d %H:%M:%S')
    elif ymd:
        return date.strftime('%Y-%m-%d')
    elif h:
        return date.strftime('%H')
    else:
        raise Exception("Неправильный запрос!")


### Получение информации из самих сообщений

#### Количество сообщений

In [ ]:
messages_count = (chat_df.groupby(['id_user'])[['id']].count().merge(users_df, left_on='id_user', right_on='id')[
                      ['username', 'id_x']].rename({'id_x': 'count'}, axis='columns')
                  .sort_values(by='count', ascending=False).reset_index(drop=True))

messages_count.head(26)

In [ ]:
date_df = chat_df['date'].map(lambda x: get_date(x, h=False))
date_df = date_df.groupby(date_df).count().rename('count')

date_df

In [ ]:
date_time_df = chat_df['date'].map(lambda x: get_date(x, ymd=False))
date_time_df = date_time_df.groupby(date_time_df).count().rename('count')

date_time_df

In [ ]:
pairs_msg = (chat_df.loc[0:,
             ['id', 'id_user', 'text', 'response_id', 'response_id_user', 'response_text', 'response_attachments_type',
              'response_attachments']])
pairs_msg = pairs_msg[pairs_msg['response_id'] != -1]
pairs_msg['count'] = pairs_msg.groupby(['response_id', 'text'])['response_id'].transform('count')
pairs_msg = pairs_msg.groupby(['response_id']).head(1)

pairs_msg = pairs_msg.merge(users_df, left_on='response_id_user', right_on='id')

pairs_msg = pairs_msg[pairs_msg['count'] > 1]

# TODO: Проблема с парами сообщений
pairs_msg = pairs_msg[['username', 'response_text', 'text', 'response_attachments_type', 'response_attachments', 'count']]
pairs_msg = (pairs_msg.sort_values(by=['count', 'response_attachments', 'text'], ascending=[False, True, False])).reset_index(drop=True)

pairs_msg = pairs_msg.rename({'text': 'response', 'response_text': 'text',
                              'response_attachments': 'attachments', 'response_attachments_type': 'attachments_type'}, axis='columns')

pairs_msg = pairs_msg.astype({'count': int})
pairs_msg

In [ ]:
most_replied_msg = chat_df.loc[0:, ['id_user', 'response_id', 'response_id_user']]
most_replied_msg = most_replied_msg[most_replied_msg['response_id'] != -1]

most_replied_msg['count'] = most_replied_msg.groupby(['response_id_user', 'id_user'])[['response_id_user']].transform(
    'count')
most_replied_msg = most_replied_msg.groupby(['response_id_user']).head(1)
most_replied_msg = most_replied_msg[most_replied_msg['count'] > 1]

most_replied_msg = most_replied_msg.merge(users_df, left_on='response_id_user', right_on='id')
most_replied_msg = most_replied_msg.merge(users_df, left_on='id_user', right_on='id')

most_replied_msg = most_replied_msg.rename({'username_x': "replied_to", 'username_y': "replied_by"}, axis='columns')
most_replied_msg = most_replied_msg[['replied_to', 'replied_by', 'count']]
most_replied_msg = most_replied_msg.sort_values(by=['count', 'replied_to', 'replied_by'],
                                                ascending=[False, False, False]).reset_index(drop=True)

most_replied_msg


In [ ]:
stickers_df = chat_df[chat_df['attachments_type'] == 'sticker'].copy()
stickers_df['attachments'] = stickers_df['attachments'].apply(lambda x: x[0])
stickers_df

In [ ]:
stickers_df['global_count'] = stickers_df.groupby('attachments')['id'].transform('size')
stickers_df.sort_values('global_count', ascending=False)

In [ ]:
stickers_df.groupby('attachments')['id'].count().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20, 10))
ct_img = 10
columns_count = 5

for i, img_name in enumerate(stickers_df.groupby('attachments')['id'].count().sort_values(ascending=False).head(ct_img).items()):
    # print(img_name)
    plt.subplot(ct_img // columns_count, columns_count, i + 1)
    img = mpimg.imread(f"content/visual/stickers/{img_name[0]}")
    plt.yticks([])
    plt.xticks([])
    plt.title(f'{img_name[1]}')
    plt.imshow(img)

In [ ]:
stickers_df = stickers_df.merge(users_df, left_on='id_user', right_on='id', suffixes=('', '_y')).drop(['id_y', 'text'], axis=1)
stickers_df

In [ ]:
# Самые частопишущие люди
stickers_df.groupby(['username'])['id'].count().sort_values(ascending=False)

In [ ]:
ct_img = 10
columns_count = 5

for name in stickers_df.groupby(['username'])['id'].count().sort_values(ascending=False).keys()[:]:
    # print(name)
    plt.figure(figsize=(20, 10))
    plt.yticks([])
    plt.xticks([])
    plt.title(name)
    for i, img_info in enumerate(stickers_df[stickers_df['username'] == name].groupby('attachments')['id'].count().sort_values(ascending=False).head(ct_img).items()):
        # print(img_info)
        plt.subplot(ct_img // columns_count, columns_count, i + 1)
        img = mpimg.imread(f"content/visual/stickers/{img_info[0]}")
        plt.yticks([])
        plt.xticks([])
        plt.title(f'{img_info[1]}')
        plt.imshow(img)
    # print()